# JEM092 Asset Pricing - Homework 1

### Group 82780095

Petr Dvořáček, Jan Kubal, Matyáš Mattanelli

---

## Task 1

In [13]:
#Loading the necessary libraries
suppressPackageStartupMessages({
    library(quantmod)
})


In [10]:
#Loading the assigned csv file
tickers_to_download <- read.csv("data_HW1/82780095_data_download.csv")
no_of_tickers <- nrow(tickers_to_download) #Storing the number of tickers for future use

In [ ]:
#Downloading the data for Yahoo Finance
data_list <- vector("list", no_of_tickers) #Initializing an empty list for the data
for (i in 1:no_of_tickers) { 
    data_list[[i]] <- getSymbols(tickers_to_download[i, 2], src="yahoo", auto.assign = F, from = "2007-01-01", to = "2022-02-28")
}

In [ ]:
#Initializing an empty matrix for the data
dates <- seq(from = as.Date("2007-01-01"), to = as.Date("2022-02-28"), by = "day")
df <- matrix(nrow = length(dates), ncol = 6)
colnames(df) <- c("Firm", "Date", "Adj_Close_Price", "Volume", "Market_Cap", "BV_per_Share")
df$Firm <- rep(tickers_to_download[, 1], length(dates))
df$Date <- dates